In [1]:
function calculate_branching(x)
    arc = 0
    found = false

    for i in 1:size(x,1) # avoir une variables serait bien pour la taille
        if (x[i]-floor(x[i]))>ϵ && (ceil(x[i])-x[i]>ϵ) # prend le premier qui arrive ??
            arc=i
            found=true
            break
        end       
    
        found && break
    end
    return arc
end



calculate_branching (generic function with 1 method)

In [ ]:
function solve_BP(initialColumn = false)

    global column_pool = Array{Array{Int,1},1}()
    # this is a 3-dimensional array
    # the first index is for machines
    # column_pool[i] contains the patterns for machine i which is a 2-dimensional array
    # column_pool[i][j] is a pattern  (1-dimensional array = vector)
    # pattern vectors are  such that:
    #   -the first element is the objective value contribution
    #   -the second to n+1 st elements are 1 or 0 depending on whether nth item is included in the pattern or not
    #   -the last element is the contribution to the convexity constraint (it is always 1)

    if initialColumn
        SPobj,column = initial_column()

        for i in 1:size(column,1)
            column[i] = round(Int64,column[i])
        end
        push!(column_pool,vcat(SPobj,column))
        
        global UB = sum(sort(weights, rev=true)[1:nv(pool)])
        global LB = SPobj
    else
       # set the global lower and upper bound to +,- infinity
        global UB = Inf  # If we have a heuristic solution its value can be used   
        global LB = -Inf # If we have an initial relaxation bound it can be used 
    end
    

    push!(column_pool,vcat(1,ones(Int,ne(pool)+1))) # push an artificial pattern
    push!(column_pool,vcat(0,zeros(Int,ne(pool)+1)))    # push an empty pattern
        
    global Queue = Vector{Int}() # the Queue of nodes to process is an array of integers
    global tree = Vector{TreeNode}() # the branch and price tree is an array of TreeNode objects
    
    # the branch and price tree is initialized with the root node
    # the root node is its own parent, it has no children (at the moment), its UB is infinity, 
    # there are no variables set to 0 or 1 at the root node
    push!(tree,TreeNode(0,[],Inf,[],[]))   
    push!(Queue,1)  # the indices of nodes to be processed are pushed to the Queue

    while length(Queue)>0
        println("Current list of nodes to be processed:", Queue)
        # we will process the last node in the Queue
        current = Queue[end]
        # Process_Node is the function that solves the restricted master LP
        # for the current node and returns its optimal solution
        # If the master LP is infeasible then returns []
        x = Process_Node(current)
        #println("Incompréhension dans l'assemblée $(x)\n")
        # if the current node is not infeasible and its lower
        # bound is promising then we branch
        #if size(x,1)!=0 && tree[current].lb <= UB
        if size(x,1)!=0 && tree[current].ub <= LB
            # in order to find a variable x_ij to branch on
            # we call the function calculate_branching()
            # it currently returns the first pair of (machine,job) indices
            # such that x_ij fractional
            # if no such pair is found then it returns (0,0)
            arc = calculate_branching(x)
            
            # if we have found a variable to branch on 
            # then we proceed with adding the children node
            # otherwise the optimal master LP solution is integer
            # there is no need for branching
            if arc!=0 # car arc est mis à 0 si rien ne s'est passé
                println("Two new nodes are created branching on variable x[$arc]")
                push!(tree,TreeNode(current,[],tree[current].ub,vcat(arc,tree[current].setzero),tree[current].setone))
                push!(tree[current].children,length(tree))
                push!(Queue,length(tree))
                push!(tree,TreeNode(current,[],tree[current].ub,tree[current].setzero,vcat(arc,tree[current].setone)))
                push!(tree[current].children,length(tree))
                push!(Queue,length(tree))
            else
                println("The optimal relaxation solution was integer")
                println("Feasible solution with value $(tree[current].ub) found")
            end
        else
            println("The node is either infeasible or pruned by bound")
        end
      
        
        # calculate the global upper bound as the
        # maximum of upper bounds among all currently active nodes
        global UB = tree[current].ub
        for i in Queue
            #println("---------------------------------------------------------------\n$(tree[i])\n")
            if tree[i].ub >= UB global UB=tree[i].ub end
        end
        # print the current global lower and upper bounds
        println("Current : LB=$LB,UB=$UB")
        # remove the processed node from the Queue
        deleteat!(Queue,findfirst(x -> x == current, Queue))
        # remove any nodes whose bound is not promising from the Queue
        deleteat!(Queue,unique(nodestobedeleted))
        # STOP if the optimality gap is sufficiently small
        if (2*abs(UB-LB)/abs(UB+LB))<=ϵ
            println("Optimality gap small enough : $(2*(UB-LB)/(UB+LB))")
            println("UB final = $(UB) \t LB final = $(LB)")
            break
        end
        if length(Queue) == 0
            return x
        end
    end
    println("UB final = $(UB) \t LB final = $(LB)")
end